#### 1. 크롤링을 사용한 이유는?

A. news를 수집할 수 있는 패키지가 있지만 패키지안에 있는 news 데이터에 국한되지않고, 대표 신문사 중 하나인 NBC에서 데이터를 수집함으로써 주관적인 요소가 포함되지 않고, 다양한 분야의 많은 데이터를 수집할 수 있기 때문에 크롤링을 사용했다.

#### 2. javascript를 사용한 이유는?

A. 파이썬으로도 쉽게 크롤링이 가능하지만, 매일 data를 자동으로 쉽게 크롤링 할 수 있다는 장점에서 javascript로 크롤링을 진행하였다.

#### 3. 왜 기간은 5년으로 지정했는가?

A. 10년인 경우도 있고 6개월인 경우도 있다. 기간이 짧은 경우는 계절, 시간의 흐름을 충분히 담기엔 한계가 있다고 생각한다. 또한, 코로나 이전 상황과 이후 상황도 있으므로 기간은 코로나 이후로만 잡는 것은 안좋을 것으로 생각한다. 그러므로 5년 이상은 되어야 꽤 긴 기간동안 다양한 계절과 다양한 일들에 대한 정보를 포함하고 있을 수 있다. 또한 기간이 너무 길면 현재로는 의미없는 정보들도 포함이 되어있어 불필요할 것이라 생각하여 5년이 적정하다고 생각하였다.
기존의 논문들은 기간이 짧거나 하루 뉴스 양이 상당히 작은 부분이 문제라고 생각하였으므로 기간을 길게 하고 뉴스의 양을 많이, 카테고리를 다양하게 수집한 것이 차별점이다.
- Combining Time Series and Sentiment analysis for Stock Market Forecasting, 2021 : 뉴스의 양이 적은 논문

#### 4. NBC 사이트로 정한 이유는?

A. 긴 기간동안의 데이터를 다양한 카테고리의 다양한 뉴스를 많이 추출할 수 있기 때문이다. 그리고 NBC 사이트는 미국에서 꽤 유명한 뉴스 방송사이므로 많은 사람들이 뉴스를 보는 방송사이다. 이로인해 신뢰도도 확보가 되었으므로 NBC 사이트를 사용한다.

#### 5. 날짜가 없는 데이터는 삭제한 이유는?

A. 뉴스가 주가에 영향을 주는 부분을 확인하기 위해서 날짜는 중요한 변수이다. 날짜가 없다면 데이터로 사용을 할 수 없다.

#### 6. 날짜가 없는 데이터들은 왜 생기는 것인가?

A. NBC는 지난 뉴스들을 한 부분에 모아 기록해둔다.
그 당시에는 뉴스가 생성이 되어서 기사의 제목은 현재까지 기록으로 남아있지만 현재는 기사가 사라진 경우 또는 동영상 기사인 경우이다.
추가적으로 동영상 기사인 경우, 데이터로 사용할 뉴스 본문이 없으므로 사용할 수 없다.

#### 7. 기사의 최초 작성일만 사용한 이유는?

A. 최초로 기사가 떴을 때 가장 영향을 많이 받았을 것이다. 또한 기사의 전체 내용이나 흐름이 바뀌지는 않았을 것이다. 그러므로 처음 떴을 때를 기준으로 사용하는 것이 좋다고 판단한다.

#### 8. 빈도수를 기반해 삭제한 이유는?

A. 수많은 뉴스 중 빈도수가 5개 이하인 단어의 경우는 이 단어가 데이터를 분석하는데 큰 의미가 없다고 판별한다. 불필요한 데이터를 줄이는 것이 맞다.

#### 9. title 데이터와 text 데이터를 합친 이유는?

A. 딥러닝의 input 데이터를 만들기 위해 필요한 과정이다. 또한, title과 text가 크게 다른 맥락이라고 볼 수 없다. 그러므로 같은 맥락과 같은 의미를 가진 두 가지를 합쳐 한 벡터로 생성해 분석하는 것은 합당한 일이라고 판단한다. 

#### 10. 왜 title데이터와 text 데이터를 사용하는 것인가?

A. 기존의 논문들에는 title 데이터만 사용하는 경우가 많았다. 본문을 사용하여 기존 논문들과의 차별화를 두었다.
- Combining Time Series and Sentiment analysis for Stock Market Forecasting, 2021   : title만 사용한 논문
- Stock prediction using combination of BERT sentiment Analysis and macro economy index : headline만 사용한 점을 개선점으로 언급

In [1]:
import pandas as pd
import numpy as np
import regex as re
    
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


from multiprocessing import Pool

import warnings
warnings.filterwarnings('ignore')
import itertools

2022-11-24 20:28:37.179281: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-24 20:28:37.348317: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-24 20:28:37.355652: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-24 20:28:37.355673: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
import datetime

In [3]:
year = list(map(str, list(range(2017,2023))))
month = list(map(str, list(range(1,13))))

for i in range(0,9):
    month[i] = "0"+month[i]

year_month = [0]*(12*5+4)

k=0
for i in range(len(year)):
    for j in range(len(month)):
        if k < len(year_month):
            year_month[k] = year[i]+"_"+month[j]
            k = k+1
        else:
            k=k+1

In [4]:
## 소문자 변환 - tit, txt에 적용(text, title을 토큰화한 리스트)
def upper_to_lower_line(line):  ## 한 리스트의 한 문장씩 불러와서 upper_to_lower_word 적용
    line = list(map(upper_to_lower_word,line))
    return line

def upper_to_lower_word(word):  ## string 형식, 한 문장의 한 단어씩 불러와서 대문자 변환   The -> the, THE -> THE
    word = re.sub("^[A-Z]{1}[a-z]*$",word.lower(),word)  ## 정규표현식에 맞는 단어만 lower 적용
    return word





## 표제어 추출 - tit, txt에 적용
def lemmatization_line(line):   ## 한 리스트의 한 문장씩 불러와서 lemmatization_line 적용
    line = list(map(lemmatization_word, line))
    return line

def lemmatization_word(data):  ## string 형식, 한 문장의 한 단어씩 불러와서 표제어 추출
    n=WordNetLemmatizer()
    data = n.lemmatize(data)
    return data





## 불용어 제거 - tit, txt에 적용
def del_stopword(line):              ## 한 리스트의 한 문장씩 불러와서 dir_stopword_produce 적용
    dir_stop_words = stopwords.words('english')  ## 불용어 사전
    
    line_stopwords_intersection = list(set(line)& set(dir_stop_words))   ## 각 문장과 불용어 사전에 동시에 있는 단어 추출
    
    # 각 문장마다 불용어 사전과 교집합인 사전 생성
    
    # 각 문장마다 교집합 사전에 해당하지 않는 값만 추출
    line = difference(line, line_stopwords_intersection)  
    
    return line


def difference(line, line_stopwords_intersection):      ## 각 문장, 각 문장과 불용어 사전의 교집합 입력
    line = [i for i in line if i not in line_stopwords_intersection]  ## 불용어 사전에 해당하지 않는 단어만 추출
    return line






dir = "/home/whfhrs3260/data/"

def loading(data):

    ## ----------------------데이터 파일 불러오기--------------------------------------------------
    f = open(dir+str(data)+".txt","r", encoding = "UTF-8")
    datas = f.readlines()
    f.close()
    
    
    
    
    ## ----------------------데이터 저장--------------------------------------------------
    n =len(datas)
    m=0
    original_data = [0]*n
    for line in datas:
        original_data[m] = line.strip()   ## 한줄씩 읽기
        m+=1
        
    for k in range(n):
        original_data[k] = original_data[k].split(" --- ")  ## 구분자를 이용해 구분
    
    original_data = pd.DataFrame(original_data)
    original_data.columns = ["category", "date", "title", "text", "author"]  # specify the column name
    original_data.index = list(range(len(original_data)))
    
    
    
    
    
    ## ----------------------데이터 구조 변경----------------------------------------------------
    data_title_indx = np.array(np.where(original_data.iloc[:,3].isnull())[0])
    data_title = original_data.iloc[data_title_indx,:]
    data_text_indx = np.array(np.where(original_data.iloc[:,4].isnull())[0])
    data_text = original_data.iloc[np.setdiff1d(data_text_indx, data_title_indx),:]
    
    
    
    # case 1. title만 있는 경우 변경
    data_1_indx = np.intersect1d(data_title_indx,original_data[original_data["title"] == " ---"].index)
    data_1 = original_data.iloc[data_1_indx,:] ## case 1에 맞는 데이터
    data_1.title = data_1.date
    data_1.loc[:,["category", "date"]] = None
    original_data.iloc[data_1_indx,:] = data_1
    
    
    
    # case 2. title과 text만 있는 경우 변경
    data_2_indx = np.setdiff1d(data_title_indx, data_1_indx)
    data_2 = original_data.iloc[data_2_indx,:]
    data_2.text = data_2.title
    data_2.title = data_2.date
    data_2.loc[:,["category", "date"]] = None
    original_data.iloc[data_2_indx,:] = data_2
    
    
    
    # case 3. title, text, author만 있는 경우
    data_3_indx = np.intersect1d(data_text_indx,original_data[original_data["category"] == "--- "].index)
    data_3 =original_data.iloc[data_3_indx,:]
    data_3.author = data_3.text
    data_3.text = data_3.title
    data_3.title = data_3.date
    data_3.loc[:,["category", "date"]] = None
    original_data.iloc[data_3_indx,:] = data_3

    
    
    # 기사 최초 작성일만 추출
    original_data.date = original_data.date.str.split("/").str[0]

    
    
    
    
    ## ----------------------불필요한 데이터 제거----------------------------------------------------
    date_null_ind = np.array(np.where(original_data.iloc[:,[1]].isnull())[0])
    date_black_ind = np.array(np.where(original_data.date=="")[0])
    
    del_date_ind = np.concatenate((date_null_ind, date_black_ind)) # date 열이 None인 행의 인덱스 추출
    original_data = original_data.drop(del_date_ind)  # 인덱스에 해당하는 행 삭제 즉, date 열이 None인 행 삭제
    
    
    
    
    
    ## ----------------------데이터 시간 형식 변경----------------------------------------------------
    ## 원래 date : Jan. 1, 2021, 12:23 AM UTC 
    ## 가공된 date : 2021-01-01 00:23:00
    original_data.date = original_data.date.apply(lambda x:x.strip(" UTC\xa0")) ## data의 date 열에서 UTC\xa0이나 UTC에 해당하는 부분 삭제
    original_data.date = original_data.date.apply(lambda x:x.strip(" UTC"))

    original_data.date = original_data.date.apply(lambda x: pd.to_datetime(x, errors="ignore"))  ## datetime 형식으로 변경
    
    # 기준 이전의 날짜는 자르기
    original_data = original_data[original_data.date>="2012-01-01"]
    original_data = original_data[original_data.date<"2022-05-01"]
    
    
    ## ---------------------text와 title 결합--------------------
    original_data["title_text"] = original_data["title"]+" " +original_data["text"]
    
    
    
    ## ----------------------구두점 제거----------------------------------------------------
    fullstop = re.compile(r'[,—"“”‘’\'-?:!;\\]')
    original_data.title_text = original_data.title_text.apply(lambda x: fullstop.sub(" ",x))  ## title에서 제거
    
           
    
    ## ----------------------토큰화----------------------------------------------------
    tit_txt_combination = list(map(word_tokenize, original_data.title_text))
    
    
    
    
    ## ----------------------대문자 소문자 변환----------------------------------------------------
    tit_txt_combination = list(map(upper_to_lower_line, tit_txt_combination)) ## 첫번째 문자만 대문자인 경우 소문자로 변환
    
    
    
    
    ## ----------------------표제어 추출----------------------------------------------------
    tit_txt_combination = list(map(lemmatization_line, tit_txt_combination))
   
    
    
    ## ----------------------불용어 제거----------------------------------------------------
    tit_txt_combination = list(map(del_stopword, tit_txt_combination))
    
    
    
    ## ----------------------데이터 프레임으로 형식 변환----------------------------------------------------
    tit_txt_combination = pd.DataFrame(tit_txt_combination)    # 데이터를 정리하기 위해서 변환이 필요
    
    return [original_data, tit_txt_combination]


    
def list_chunk(lst, num_cores):   ## num_cores에 맞게 df를 분리하는 함수  1,2,3,4 -> 코어가 2개인 경우 [1,2],[3,4]로 분류
    return [lst[i:i+num_cores] for i in range(0, len(lst), num_cores)]              ##   각 코어어 할당해주기 위해 필요


def type_conversion(df):   ## 한 코어가 실행한 n개의 데이터 프레임을 1개로 합치는 함수
    
    original_data = pd.DataFrame()
    tit_txt_combination = pd.DataFrame()
    
    for j in df:
        original_data_, tit_txt_combination_ = j
        original_data = pd.concat([original_data,original_data_])
        tit_txt_combination = pd.concat([tit_txt_combination,tit_txt_combination_])
    return [original_data, tit_txt_combination]


def parallel(df, func, num_cores):
    
    df_split = list_chunk(df,num_cores)  ## num_cores에 맞게 데이터 분리

    pool = Pool(num_cores) ## process에 분배하여 함수 실행의 병렬처리를 도와줌
    
    def pool_map(df_split):
        df = pool.map(loading, df_split)          ## 위에서 나눈 df_split을 각 process에 할당하여 실행
        return df
    
    
    df = list(map(pool_map, df_split))
    
    df = list(map(type_conversion,df))  ## 각각의 코어가 맡은 데이터프레임을 합침 즉, 1개의 코어가 2개의 데이터프레임을 실행했다면? 1개의 데이터프레임으로 합침

    ## 각 코어들이 만든 데이터 프레임도 합침
    original_data, tit_txt_combination = type_conversion(df)
    
    pool.close()
    pool.join()

    return original_data,tit_txt_combination

def main():
    df = year_month
    num_cores = 28
    original_data, tit_txt_combination = parallel(df, loading, num_cores)
    return original_data, tit_txt_combination

if __name__ == "__main__":
    start = datetime.datetime.now()
    original_data, tit_txt_combination = main()
    print(datetime.datetime.now()-start)

0:03:38.068907


In [5]:
start = datetime.datetime.now()
tit_txt_combination = tit_txt_combination.where(pd.notnull(tit_txt_combination), None)
print(datetime.datetime.now()-start)

0:02:01.063026


In [7]:
## 빈도수 기반 사전 생성
def del_list_None(lst):
    lst = list(filter(None, lst))
    return lst

start = datetime.datetime.now()  ## 시작 시간

tit_txt_combination = tit_txt_combination.values.tolist()  ## 데이터프레임 -> list
tit_txt_combination = list(map(del_list_None, tit_txt_combination))   ## 변환하면서 생긴 None 삭제

frequent_tit_txt = pd.DataFrame(pd.Series(list(itertools.chain(*tit_txt_combination))).value_counts())   ## tit에 있는 단어들의 빈도수 추출
tit_txt_less_frequent_word = list(frequent_tit_txt[4 >= frequent_tit_txt[0]].index)     ## 빈도가 4 이하인 단어를 추출

print(datetime.datetime.now()-start)  # 끝나는 시간

0:02:57.329801


In [8]:
from multiprocessing import Pool

def del_less_frequent_tit_txt(line):  ## 리스트의 한 line  ["a","b","c"]
    line_frequent_intersection = list(set(line)& set(tit_txt_less_frequent_word))   ## 불용어 사전과 동일하게 각 line과 불용어의 교집합 사전을 만들어 해당되는 값 제거
    data = difference(line, line_frequent_intersection)
    return data

def work_func(tit_txt_combination):
    tit_txt_combination = list(map(del_list_None, tit_txt_combination))  
    tit_txt_combination = list(map(del_less_frequent_tit_txt, tit_txt_combination))
    return tit_txt_combination

def main():
    num_cores = 28
    df = tit_txt_combination
    df_split = np.array_split(df,num_cores)
    pool = Pool(num_cores)
    df = pool.map(work_func, df_split)
    pool.close()
    pool.join()

    return df

if __name__ == "__main__":
    start = datetime.datetime.now()
    tit_txt_combination = main()
    print(datetime.datetime.now()-start)

0:04:04.617430


In [10]:
for i in range(len(tit_txt_combination)-1):
    i += 1
    tit_txt_combination[0].extend(tit_txt_combination[i])
tit_txt_combination = tit_txt_combination[0]

In [11]:
start = datetime.datetime.now()
tit_txt_combination = pd.DataFrame(tit_txt_combination)
print(datetime.datetime.now()-start)

0:03:30.082053


In [12]:
dir = "/home/whfhrs3260/csv_data/"

In [13]:
start = datetime.datetime.now()
original_data.to_pickle(dir+"original_data.pkl")
print(datetime.datetime.now()-start)

0:00:01.360573


In [14]:
start = datetime.datetime.now()
tit_txt_combination.to_pickle(dir+"tit_txt_combination_4years.pkl")
print(datetime.datetime.now()-start)

0:00:36.690935


In [16]:
original_data

,category,date,title,text,author,title_text
0,U.S. news,2017-01-01 00:20:00,Texas Judge Halts Federal Transgender Protections,"AUSTIN, Texas — A federal judge in Texas has o...",The Associated Press,Texas Judge Halts Federal Transgender Protecti...
1,World,2017-01-01 01:08:00,At Least 39 Killed in Shooting at Turkey Night...,,NBC News,At Least Killed in Shooting at Turkey Night...
2,World,2017-01-01 02:31:00,President Obama Briefed on Turkey Attack,,NBC News,President Obama Briefed on Turkey Attack
3,TV,2017-01-01 03:56:00,"Actor William Christopher, ""M*A*S*H"" Chaplain,...","William Christopher, who played the unassuming...",Reuters,Actor William Christopher M A S H Chaplain ...
4,World,2016-12-31 14:07:00,"Hello, 2017! Cities Around the World Ring in S...",Revelers rang in 2017 across the globe with da...,Alastair JamiesonErik OrtizAndy Eckardt,Hello Cities Around the World Ring in S...
...,...,...,...,...,...,...
1826,Pop Culture News,2022-04-30 16:17:00,Veteran TV actress Jossara Jinaro dies of canc...,"Veteran actress Jossara Jinaro, known for her ...",Nicole Acevedo,Veteran TV actress Jossara Jinaro dies of canc...
1827,Russia-Ukraine Conflict,2022-04-30 17:32:00,"Angelina Jolie visits children, doctors in Ukr...","Actor Angelina Jolie arrived in Lviv, Ukraine ...",Diana DasrathMinyvonne Burke,Angelina Jolie visits children doctors in Ukr...
1828,U.S. news,2022-04-30 19:34:00,"Teacher under fire for cotton, cuffs in class ...",Rochester school officials are investigating a...,Associated Press,Teacher under fire for cotton cuffs in class ...
1829,Culture Matters,2022-04-30 19:22:00,Bill Murray speaks on accusations of inappropr...,Actor Bill Murray opened up about what led to ...,Minyvonne Burke,Bill Murray speaks on accusations of inappropr...


In [15]:
tit_txt_combination

,0,1,2,3,4,5,6,7,8,9,...,16264,16265,16266,16267,16268,16269,16270,16271,16272,16273
0,texas,judge,halt,federal,transgender,protection,AUSTIN,texas,federal,judge,...,None,None,None,None,None,None,None,None,None,None
1,least,killed,shooting,turkey,nightclub,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,president,obama,briefed,turkey,attack,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,actor,william,christopher,h,chaplain,dead,william,christopher,played,unassuming,...,None,None,None,None,None,None,None,None,None,None
4,hello,city,around,world,ring,start,new,year,reveler,rang,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93766,veteran,TV,actress,jinaro,dy,cancer,age,family,said,veteran,...,None,None,None,None,None,None,None,None,None,None
93767,angelina,jolie,visit,child,doctor,ukraine,actor,angelina,jolie,arrived,...,None,None,None,None,None,None,None,None,None,None
93768,teacher,fire,cotton,cuff,class,slavery,rochester,school,official,investigating,...,None,None,None,None,None,None,None,None,None,None
93769,bill,murray,speaks,accusation,inappropriate,behavior,led,suspension,mortal,actor,...,None,None,None,None,None,None,None,None,None,None


1. 타이틀, 본문을 왜 결합 사용: 딥러닝 모형
2. 타이틀에 있는 단어는 거의 본문에서 다시 사용하지 않는가? 다른 단어지만 의미는 같은 단어?
3. 다른 전처리 과정에 통합해야 할 듯